In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkConf

In [2]:
spark = SparkSession.builder.getOrCreate()
df = spark.read.csv("lab2data/all_order_products.csv", header='True', inferSchema='True')
#df.select("order_id", "product_id").show()

In [3]:
order2products = df.groupBy("order_id").agg(collect_list("product_id").alias('items'))

In [4]:
order2products.count()

3346083

In [5]:
#order2products.show()
order2products=order2products.filter(size('items')>=70)

In [6]:
order2products.count()

315

In [7]:
order2products=order2products.filter(size('items')<=90)

In [8]:
order2products.count()

260

In [9]:
order2products.select("order_id", "items",size("items").alias("size")).show()

+--------+--------------------+----+
|order_id|               items|size|
+--------+--------------------+----+
|   28332|[39236, 19348, 57...|  71|
|   70299|[33754, 24852, 47...|  83|
|  181716|[49374, 16830, 30...|  70|
|  267132|[5161, 36735, 903...|  73|
|  341133|[28358, 36554, 42...|  76|
|  348609|[14704, 38456, 49...|  76|
|  374902|[401, 33511, 1184...|  81|
|  388234|[26416, 145, 3803...|  70|
|  402458|[14947, 35221, 44...|  70|
|  484715|[39558, 16349, 33...|  72|
|  488652|[47301, 6828, 197...|  74|
|  513607|[24852, 116, 2390...|  81|
|  521291|[36070, 24852, 14...|  72|
|  542925|[24852, 45840, 16...|  71|
|  544549|[10031, 13573, 36...|  74|
|  579891|[10685, 38734, 20...|  74|
|  583387|[10031, 13573, 23...|  70|
|  653280|[12576, 43654, 24...|  72|
|  672082|[37849, 46780, 21...|  78|
|  678116|[24852, 3318, 116...|  70|
+--------+--------------------+----+
only showing top 20 rows



In [10]:
# sum of product's order
product2order = df.groupBy("product_id").agg(count("*").alias("order_sum")).sort(desc("order_sum"))

In [11]:
product2order=product2order.filter(product2order.order_sum>100)
product2order.select(mean("order_sum").alias("mean"),
                    avg("order_sum").alias("avg")).show()

+------------------+------------------+
|              mean|               avg|
+------------------+------------------+
|1614.2502085070892|1614.2502085070892|
+------------------+------------------+



In [12]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="items",minSupport=0.1, minConfidence=0.1)
model = fpGrowth.fit(order2products)

In [13]:
model.freqItemsets.count()

42

In [14]:
model.freqItemsets.show(20)

+-------+----+
|  items|freq|
+-------+----+
|[25718]|  27|
| [9076]|  33|
|[47209]|  55|
|[46979]|  26|
|[39928]|  30|
|[16797]|  40|
|[33754]|  28|
| [5876]|  36|
|[13176]|  78|
|[19677]|  29|
|[49683]|  39|
|[36735]|  27|
|[27344]|  36|
|[24852]|  70|
|[41844]|  27|
|[44359]|  32|
| [4920]|  53|
|[45007]|  27|
|[44632]|  34|
|[21137]|  66|
+-------+----+
only showing top 20 rows



In [15]:
model.associationRules.show()

+----------+----------+-------------------+------------------+------------------+
|antecedent|consequent|         confidence|              lift|           support|
+----------+----------+-------------------+------------------+------------------+
|   [13176]|   [21137]|0.44871794871794873|1.7676767676767677|0.1346153846153846|
|   [21137]|   [13176]| 0.5303030303030303|1.7676767676767677|0.1346153846153846|
+----------+----------+-------------------+------------------+------------------+

